In [4]:
import torch
import od3d.io
from od3d.cv.geometry.mesh import Meshes
from od3d.cv.visual.show import  show_scene2d , show_scene,show_imgs
from od3d.cv.cluster.embed import tsne, pca


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [14]:
ckpt_path2 = "/home/heisenberg/Thesis_repo/exps_remote/03-07_14-39-24_Pascal3D_NeMo_torque/nemo.ckpt" # moving average without head
ckpt_path1 = "/home/heisenberg/Thesis_repo/exps_remote/03-10_22-30-31_Pascal3D_NeMo_torque/nemo.ckpt" # moving average with head with mesh feats updated
#ckpt_path2 = "/home/heisenberg/Thesis_repo/exps_remote/03-09_18-59-01_Pascal3D_NeMo_torque/nemo.ckpt" # loss gradient with head

In [15]:
checkpoint1 = torch.load(ckpt_path1, map_location='cpu')
checkpoint2 = torch.load(ckpt_path2, map_location='cpu')

In [19]:
checkpoint2['clutter_feats']

Parameter containing:
tensor([[-8.5512e-02, -3.9336e-02, -2.4450e-01,  1.3426e-02, -1.1178e-02,
         -5.3613e-02, -5.7601e-02, -3.0369e-02, -9.9373e-02, -3.3583e-02,
         -1.5454e-01,  2.0951e-01,  2.8824e-02, -1.7034e-02,  1.1899e-01,
          1.0931e-01, -1.1750e-02,  1.1254e-01,  1.3274e-01,  9.8071e-02,
         -4.5697e-02, -1.3173e-01, -6.6995e-02, -1.1997e-01,  6.4449e-03,
         -6.8495e-04,  1.7543e-01,  6.3605e-02, -7.2086e-03, -7.7431e-02,
          1.2491e-01, -2.6627e-02, -5.5167e-02, -3.0357e-02, -8.4027e-02,
         -7.4532e-02, -6.8496e-02, -4.6596e-03,  5.0260e-02,  1.4011e-01,
          1.3059e-03,  4.1912e-02,  1.5120e-01,  6.5882e-02, -1.6699e-02,
          9.4812e-02, -1.3522e-01, -6.9857e-02, -1.6983e-02,  3.5363e-02,
          3.6231e-02, -7.6749e-03,  8.2965e-02, -1.0286e-01,  1.0325e-02,
          4.4971e-02, -5.0858e-02, -2.9879e-02, -5.0576e-02,  8.3683e-02,
          2.6235e-02,  1.1193e-02,  4.3740e-02, -1.0009e-01, -3.0931e-03,
         -9.0023

In [17]:
torch.equal(checkpoint1['clutter_feats'],checkpoint2['clutter_feats'])

True

In [ ]:
tensor_mesh = torch.nn.Parameter(torch.rand(3, 3))
tensor_new = torch.rand(3, 3)
tensor3_bank = tensor_mesh
tensor3_bank.data = tensor_new



In [ ]:
tensor3_bank

In [ ]:
tensor_new

In [ ]:
tensor_mesh

In [ ]:
benchmark = "pascal3d_nemo_dino"
ckpt_path = "/home/heisenberg/Thesis_repo/exps_remote/03-07_14-39-24_Pascal3D_NeMo_torque/nemo.ckpt"

In [ ]:
checkpoint = torch.load(ckpt_path)

In [ ]:
cfgs = [od3d.io.load_hierarchical_config(benchmark=benchmark, platform="local")]

In [ ]:
methods_cfgs = []
for i, cfg in enumerate(cfgs):
        #logger.info(f'{i} of {len(cfgs)}')
        methods_keys = cfg.method.keys()
        for key in methods_keys:
            method_cfg = cfg.copy()
            method_cfg.method = cfg.method[key]
            method_cfg_exists = False
            for prev_method_cfg in methods_cfgs:
                if method_cfg == prev_method_cfg:
                    method_cfg_exists = True
            if not method_cfg_exists:
                methods_cfgs.append(method_cfg)

print(f"{len(methods_cfgs)} configs with single method.")

In [ ]:
methods_cfgs
method_config = methods_cfgs[0].method

In [ ]:
fpaths_meshes = [method_config.fpaths_meshes[cls] for cls in method_config.categories]
meshes = Meshes.load_from_files(fpaths_meshes=fpaths_meshes)
    

In [ ]:
meshes.set_feats_cat(checkpoint['meshes_feats'])

In [ ]:
cat_borders = torch.cat([torch.LongTensor([0]).to(device=meshes.device), torch.cumsum(torch.tensor(meshes.verts_counts),dim=0)], dim=0)
feats_all_cats =[]
feats_all_colors =[]
for i, cat in enumerate(method_config.categories):
    print(f"{cat} has {meshes.verts_counts[i]} vertices and {meshes.faces_counts[i]} faces.")
    #print(f"Feats for {cat} are {len(meshes.feats[cat_borders[i]:cat_borders[i+1]])}.")
    feats_all_cats.extend(meshes.feats[cat_borders[i]:cat_borders[i+1]])
    feats_all_colors.extend( i*torch.ones(meshes.verts_counts[i], dtype=torch.long))


In [ ]:
feats_all_colors[cat_borders[1]:cat_borders[2]]

In [ ]:
meshes.feats.shape

In [ ]:
feats_all_cats_tsne = pca(meshes.feats, C=3)


In [ ]:
show_scene2d([feats_all_cats_tsne,feats_all_cats_tsne], pts2d_colors=[torch.tensor(feats_all_colors),torch.tensor(feats_all_colors)])

In [ ]:
show_scene(pts3d=[feats_all_cats_tsne[:]],pts3d_colors=[torch.tensor(feats_all_colors)])